In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
from dlc_practical_prologue import *
import matplotlib.pyplot as plt

> 0 if first > second else 1

In [2]:
train_input, train_target, test_input, test_target = load_data()

* Using MNIST
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


In [3]:
train_input, train_target, train_classes, test_input, test_target, test_classes = generate_pair_sets(1000)

In [320]:
train_target.shape

torch.Size([1000])

In [321]:
train_classes.shape

torch.Size([1000, 2])

In [119]:
nb_hidden = 20
mini_batch_size = 100

In [398]:
def train_model(model, train_input, train_target, mini_batch_size):
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=1e-3)
    for e in range(25):
        sum_loss = 0
        for b in range(0, train_input.size(0), mini_batch_size):
            optimizer.zero_grad()
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            sum_loss = sum_loss + loss.item()
            loss.backward()
            optimizer.step()
           
        print(e, sum_loss)

## Baseline : A Naive Methods

A first methods is two predict the class of the two digit and then mannualy compare them. To do this we use a adapteded version of LeNet.

In [400]:
class LeNetLike(nn.Module):
    def __init__(self):
        super(LeNetLike, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size = 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(6, 16, kernel_size = 3),
            nn.ReLU()
)
        self.classifier = nn.Sequential(
            nn.Linear(256, 120),
            nn.ReLU(inplace=True),
            nn.Linear(120, 84),
            nn.ReLU(inplace=True),
            nn.Linear(84, 10)
)
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


In [413]:
# Cut the input and target in two
train_1 = train_input[:,0,:,:].unsqueeze(1)
train_2 =  train_input[:,1,:,:].unsqueeze(1)
train = torch.cat((train_1, train_2), 0)
target = torch.cat((train_classes[:,0],train_classes[:,1] ), 0)
target_one_hot = convert_to_one_hot_labels(train, target)
model = LeNetLike() 

In [457]:
test_target.shape

torch.Size([1000])

In [408]:
# train model
train_model(model, train, target_one_hot, 100)

0 19.38427299261093
1 4.835545226931572
2 3.888506919145584
3 3.380842387676239
4 3.0548028498888016
5 2.821824789047241
6 2.642517887055874
7 2.4997337386012077
8 2.3841254115104675
9 2.288592331111431
10 2.2070624083280563
11 2.136654570698738
12 2.0754651948809624
13 2.020949475467205
14 1.9723383486270905
15 1.9285875782370567
16 1.8891212940216064
17 1.8531165719032288
18 1.8198898509144783
19 1.7893921211361885
20 1.7611387446522713
21 1.7350005581974983
22 1.7105277925729752
23 1.687525324523449
24 1.66590366512537


In [431]:
preds[0].argmax()

tensor(1)

In [450]:
# TODO: use test
# Compute predictions over 1000 pairs
pair_pred = []
for im1,im2 in zip(train_1,train_2):
    preds = model(torch.cat((im1.unsqueeze(0),im2.unsqueeze(0)),0))
    pair_pred.append((preds[0].argmax() > preds[1].argmax()).long().item())
pair_pred = torch.Tensor(pair_pred)
accuracy(pair_pred,targets)

In [455]:
target_test.shape

torch.Size([2000])

## Accuracy

In [391]:
input_test = torch.cat((test_input[:,0,:,:], test_input[:,1,:,:] ), 0).unsqueeze(1)
target_test = torch.cat((test_classes[:,0],test_classes[:,1] ), 0)
target_test_one_hot = convert_to_one_hot_labels(input_test, target_test)

In [388]:
def accuracy_one_hot(model, input_, target):
    preds = model(input_)
    return (preds.argmax(axis=1) == target.argmax(axis=1)).long().sum().item() / target.shape[0]

In [453]:
def accuracy(preds,targets):
    return (preds == targets).long().sum().item()/len(preds)

In [393]:
accuracy_one_hot(model, input_test, target_test_one_hot)

0.744